In [1]:
# first set global root data directory
import pylabs
pylabs.datadir.target = 'jaba'
from pathlib import *
import pandas as pd
import numpy as np
from pylabs.io.mixed import df2h5, h52df, get_h5_keys
from pylabs.utils import *
from pylabs.projects.acdc.file_names import project, Optsd
import qgrid
qgrid.enable
import ipywidgets as widgets
fs = Path(getnetworkdataroot())
opts = Optsd()
style = {'description_width': 'initial'}
input_selkey = widgets.Dropdown(options=list(sorted(get_h5_keys(opts.info_fname, key='auto_qc'))[-19:]), disabled=False,
    description='Subject DWI to qc:', style=style)
input_selkey.layout.width = '400px'
input_selkey

setting root data directory to jaba.
setting root data directory to jaba.
setting root data directory to jaba.
setting root data directory to jaba.
/brainstudio/data
setting root data directory to jaba.
/brainstudio/data
setting root data directory to jaba.
setting root data directory to jaba.


RHJvcGRvd24oZGVzY3JpcHRpb249dSdTdWJqZWN0IERXSSB0byBxYzonLCBsYXlvdXQ9TGF5b3V0KHdpZHRoPXUnNDAwcHgnKSwgb3B0aW9ucz0oJy9zdWItYWNkYzEwNC9zZXMtMS9kd2kvYXXigKY=


In [2]:
pick = {'subj': input_selkey.value.split('/')[1], 'session': input_selkey.value.split('/')[2], }
# new h52df function makes all cols numeric. yes!
all_dwi_auto_qc = h52df(opts.info_fname, input_selkey.value)
pick['dwi_fname'] = all_dwi_auto_qc.loc[0, 'dwi_fname']
# todo: make python scriptlet to load files into FSLeyes
# fslv_names = ' '.join([x + ' -b 1,200' for x in all_dwi_auto_qc.loc[0, ['dwi_fname', 'topup_fname', 'topdn_fname']].values])
# with WorkingContext(str(Path(all_dwi_auto_qc.loc[0, 'dwi_fname']).parent)):
#     with open('fslview_cmd.sh', 'w') as f:
#         f.write('fslview ' + fslv_names + ' &\n')

# limit to only active rows with mask
all_dwi_auto_qc.loc[all_dwi_auto_qc['itopup'] != -9999, 'itopup_visqc'] = True
all_dwi_auto_qc.loc[all_dwi_auto_qc['itopdn'] != -9999, 'itopdn_visqc'] = True
all_dwi_auto_qc.loc[all_dwi_auto_qc['itopup'] == -9999, 'itopup_visqc'] = False
all_dwi_auto_qc.loc[all_dwi_auto_qc['itopdn'] == -9999, 'itopdn_visqc'] = False
all_dwi_auto_qc.loc[:, 'dwi_visqc'] = True
with WorkingContext(str(Path(pick['dwi_fname']).parent/'qc')):  
    topup_gvolspng = open('{subj}_{session}_topup8b0-qc_good_plot.png'.format(**pick), 'rb').read()
    topup_bvolspng = open('{subj}_{session}_topup8b0-qc_bad_plot.png'.format(**pick), 'rb').read()
    topdn_gvolspng = open('{subj}_{session}_topdn7b0-qc_good_plot.png'.format(**pick), 'rb').read()
    topdn_bvolspng = open('{subj}_{session}_topdn7b0-qc_bad_plot.png'.format(**pick), 'rb').read()
    dwi800_gvolspng = open('{subj}_{session}_b800-qc_good_plot.png'.format(**pick), 'rb').read()
    dwi800_bvolspng= open('{subj}_{session}_b800-qc_bad_plot.png'.format(**pick), 'rb').read()
    dwi2000_gvolspng = open('{subj}_{session}_b2000-qc_good_plot.png'.format(**pick), 'rb').read()
    dwi2000_bvolspng= open('{subj}_{session}_b2000-qc_bad_plot.png'.format(**pick), 'rb').read()

dwicols = [u'bvals', u'auto_dwi_vol_idx', u'auto_dwi_qc', 'dwi_visqc', u'x_bvec', u'y_bvec', u'z_bvec']
topudcols = [u'itopup', u'alltopup_idx', u'topup_qc', 'itopup_visqc', u'itopdn', u'topdn_qc', 'itopdn_visqc',]
dwi_auto_qc = all_dwi_auto_qc[dwicols]
topud_auto_qc = all_dwi_auto_qc[topudcols]
dwi_w = qgrid.show_grid(dwi_auto_qc, show_toolbar=False)
topupdn_w = qgrid.show_grid(topud_auto_qc, show_toolbar=False)
pick

{'dwi_fname': '/brainstudio/data/acdc/sub-acdc104/ses-2/dwi/sub-acdc104_ses-2_dwi-topup_64dir-3sh-800-2000_1.nii',
 'session': 'ses-2',
 'subj': 'sub-acdc104'}

In [3]:
items = [widgets.Image(description='dwi b2000 bad vols', value=dwi2000_bvolspng,    format='png',    width=500,    height=600),
        widgets.Image(description='dwi b2000 good vols', value=dwi2000_gvolspng,    format='png',    width=500,    height=600),]
#items_layout = [widgets.Layout(flex-basis='50%', ), widgets.Layout(flex-basis='50%', widgets.Layout(flex-basis='100%',)]
box_layout = widgets.Layout(display='flex', align_items='stretch', border='solid', width='100%')
top_box = widgets.HBox(children=items)
bottom_box = widgets.HBox(children=[dwi_w])
dwi_box = widgets.Box([top_box, bottom_box], layout=box_layout)
dwi_box

Qm94KGNoaWxkcmVuPShIQm94KGNoaWxkcmVuPShJbWFnZSh2YWx1ZT0nXHg4OVBOR1xyXG5ceDFhXG5ceDAwXHgwMFx4MDBccklIRFJceDAwXHgwMFx4MDRceGIwXHgwMFx4MDBceDAzIFx4MDjigKY=


In [4]:
items = [widgets.Image(description='topup bad vols', value=topup_bvolspng,    format='png',    width=400,    height=600),
        #widgets.Image(description='topup good vols', value=topup_gvolspng,    format='png',    width=400,    height=600),
        widgets.Image(description='topdn bad vols', value=topdn_bvolspng,    format='png',    width=400,    height=600),
        #widgets.Image(description='topdn good vols', value=topdn_gvolspng,    format='png',    width=400,    height=600),
]
box_layout = widgets.Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    border='solid',
                    width='100%')
topupdn_box = widgets.Box(children=items+[topupdn_w], layout=box_layout)
topupdn_box

Qm94KGNoaWxkcmVuPShJbWFnZSh2YWx1ZT0nXHg4OVBOR1xyXG5ceDFhXG5ceDAwXHgwMFx4MDBccklIRFJceDAwXHgwMFx4MDRceGIwXHgwMFx4MDBceDAzIFx4MDhceDAzXHgwMFx4MDBceDDigKY=


In [5]:
dwi_w.get_changed_df()
topupdn_w.get_changed_df()
all_dwi_visqc = pd.concat([dwi_w.get_changed_df(), topupdn_w.get_changed_df()], axis=1)
df2h5(all_dwi_visqc, opts.info_fname, input_selkey.value.replace('auto_qc', 'vis_qc'))

In [10]:
topupdn_w.get_changed_df()

,itopup,alltopup_idx,topup_qc,itopup_visqc,itopdn,topdn_qc,itopdn_visqc
0,0,0,0,True,0,0,True
1,1,1,0,True,1,0,True
2,2,2,0,True,2,0,True
3,3,3,0,True,3,0,True
4,4,4,0,True,4,0,True
5,5,5,0,True,5,0,True
6,6,6,0,True,6,0,True
7,-9999,7,0,False,-9999,-9999,False
8,-9999,-9999,-9999,False,-9999,-9999,False
9,-9999,-9999,-9999,False,-9999,-9999,False


In [7]:
dwi_w.get_changed_df()

,bvals,auto_dwi_vol_idx,auto_dwi_qc,dwi_visqc,x_bvec,y_bvec,z_bvec
0,0,0,0,True,0.000,0.000,0.000
1,2000,0,1,True,1.000,0.000,0.000
2,2000,1,0,True,0.000,1.000,0.000
3,2000,2,0,True,0.000,0.000,1.000
4,2000,3,0,True,-0.625,-0.777,0.076
5,2000,4,0,True,-0.745,0.623,0.237
6,2000,5,1,True,0.025,-0.095,0.995
7,2000,6,0,True,-0.853,-0.004,-0.522
8,2000,7,1,True,-0.155,0.904,-0.399
9,2000,8,1,True,-0.025,-0.716,-0.698
